In [1]:
import numpy as np
import tensorflow
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense,SimpleRNN
import pandas as pd
from sklearn.model_selection import train_test_split
import jieba
import json
import pandas as pd
import os
import keras
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from gensim.models import word2vec
import gensim.models
from sklearn.decomposition import PCA

Using TensorFlow backend.


# 定義斷詞與整理

In [3]:
# 斷詞套件
def 斷詞(news):
    df1=news
    df1['seg']=None
    df1['word_freq']=None
    n=len(news)
    alltext=[]
    stopset=set()
    stop2=['/n','']
    with open('C:\\Users\\Big data\\Desktop\\字詞貼標\\DB104-master\\stop.txt','r',encoding='ISO-8859-1') as s:
        for line in s:
            stopset.add(line.strip('\n'))
    jieba.load_userdict('C:\\Users\\Big data\\Desktop\\字詞貼標\\DB104-master\\userdict.txt')
    for i in range(n):
        seg=''
        wf={}
        text= str(df1.loc[i]['content'])
        cut=jieba.cut(text,cut_all=False)
        for j in cut:
            if j not in stopset:
                seg +=j+' '
                seg = seg.replace("\n",'')
        df1['seg'][i]=seg
        for w in seg.split(' '):
            if w not in wf:
                wf[w]=1
            else:
                wf[w]+=1
        word_list=[(k,wf[k]) for k in wf if k not in stop2 ]
        word_list.sort(key=lambda a :a[1],reverse=True)
        df1['word_freq'][i]=word_list
    return(df1)



# 斷詞整理(去除標點符號)
def 整理(df1):
    datanews = df1["seg"]
    # 轉list
    train_data = np.array(datanews)#np.ndarray()
    datanews=train_data.tolist()#list
    df1=''
    for datanew in datanews:
        df1+=str(datanew)+'\n'
    df1=df1.replace('[','').replace('\'','').replace('「 ','').replace('」 ','').replace('」 ','').replace('、 ','').replace('： ','')
    df1=df1.replace('《 ','').replace('》 ','').replace('） ','').replace('( ','').replace('／ ','').replace('， ','')
    df1=df1.replace('╱ ','').replace('！ ','').replace('？ ','').replace('（ ','').replace('。 ','').replace('； ','').replace('… ','')
    df1=df1.replace(']','').replace(' :','').replace('\u3000','').replace('\n','')
    df2=df1
    for i in range(len(df2)):
        df1=' '.join(df2.split())
    return(df1)

# 定義word2vec

In [ ]:
#訓練語意空間
#去除只出現一次的詞
size = [50,100,150]
cbow = [0,1] #(context as input, target word as output)
for x in size:
    for z in cbow:
        y = 15
        sentences = word2vec.LineSentence(r"C:\Users\Big data\Desktop\class\funcardproject\data\文章\一整大篇_乾淨.txt")
        model = word2vec.Word2Vec(sentences,size=x,window=y,min_count=1,sg = z)
        BIN =  r'C:\Users\Big data\Desktop\class\funcardproject\word2vec_model\article_social\article_social維度'+str(x)+'win'+str(y)+'cbow'+str(z)+'.bin'
        model.wv.save_word2vec_format(BIN, binary=True)

# 執行斷詞與整理


In [ ]:
#開啟要整理的檔案
news = pd.read_excel(r"C:\Users\Big data\Desktop\class\funcardproject\data\豪哥\DB104-hau\integrate\card_article\excel_file\article_social.xlsx")
#斷詞
df1=斷詞(news)
df1
news = 整理(df1)
news

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\BIGDAT~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.817 seconds.
Prefix dict has been built succesfully.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
# 儲存excel檔
news.to_excel(r"C:\Users\Big data\Desktop\class\funcardproject\data\文章\討論\article_social_斷詞完.xlsx")

In [ ]:
# 存成TXT
with open(r"C:\Users\Big data\Desktop\class\funcardproject\data\文章\討論\article_social_一整大篇.txt","w",encoding="utf-8")as f:
    f.write(str(train_data))

# 執行建模